In [17]:
import tensorflow as tf

In [18]:
# A tensor is a multidimensional array
# Tensorflow provides a rich library of operations

print(tf.add(5,6))
print(tf.add([1,2],[3,4]))
print(tf.square(7))
print(tf.reduce_sum([1,2,3,4]))

tf.Tensor(11, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(49, shape=(), dtype=int32)
tf.Tensor(10, shape=(), dtype=int32)


In [19]:
# Each tf.Tensor has a shape and a datatype
x = tf.matmul([[1]], [[2, 3]])

print(x)
print(x.shape)
print(x.dtype)
print(x.device)  # Gives the name of the device where the Tensor is hosted

tf.Tensor([[2 3]], shape=(1, 2), dtype=int32)
(1, 2)
<dtype: 'int32'>
/job:localhost/replica:0/task:0/device:CPU:0


These tensors can be used be acceleratos (such as GPUs and TPUs) and are immutable
Tensorflow operations automatically convert NumPy array to tensors and vice versa

Tensors are explicitly converted to NumPy ndarrays using their .numpy() method. These conversions are typically 
cheap since the array and tf.Tensor share the underlying memory representation, if possible. However, sharing the 
underlying representation isn't always possible since the tf.Tensor may be hosted in GPU memory while NumPy arrays 
are always backed by host memory, and the conversion involves a copy from GPU to host memory.


Refer to https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/customization/basics.ipynb for usage of devices(GPUs, CPUs, TPUs) for execution, creating datasets from your data and applying transformations to it.



In [20]:
# Keras API provides a way to define composition of computation layers (which are treated as objects)
layer = tf.keras.layers.Dense(10)
# Here the argument denotes the dimension of the output. The dimensions of the 
# can be inferred the first time the code is run or it can be specified
# if the model is complex (or for better readability)

layer = tf.keras.layers.Dense(10, input_shape=(None,5))

# Here the None means that any dimension can be present

In [21]:
# Simply call a layer to use it
layer(tf.zeros([4,5]))

<tf.Tensor: shape=(4, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [22]:
# Inspect the trainable weights and biases in a layer
layer.variables  # or layer.trainable_variables

# Also see layer.kernel and layer.bias

[<tf.Variable 'dense_6/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[ 0.4275679 ,  0.02189386,  0.61106974, -0.04479599, -0.14924341,
         -0.08539569, -0.17754924,  0.03136039,  0.05740136,  0.25770873],
        [-0.60693765,  0.21552926,  0.1718939 ,  0.11161864, -0.12448686,
          0.21869266,  0.2639873 , -0.16515723,  0.26102597,  0.5596011 ],
        [ 0.508735  ,  0.37072748, -0.50872713,  0.55213434, -0.02300453,
          0.17345607, -0.3412639 ,  0.55753416, -0.1414443 ,  0.25654   ],
        [ 0.49926347,  0.2235223 ,  0.15231335, -0.30669126, -0.44596285,
          0.15316647, -0.10717613, -0.31753978,  0.4948601 , -0.03802329],
        [ 0.43922657,  0.3738827 ,  0.16334927,  0.3153001 , -0.39035916,
          0.2433536 , -0.43327695,  0.49330324, -0.07920229,  0.45688254]],
       dtype=float32)>,
 <tf.Variable 'dense_6/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>]

## Create a model and visualize it

In [23]:
# Import keras and check Tf version
from tensorflow import keras
from tensorflow.keras import layers

tf.__version__

'2.1.0'

In [24]:
# Create a model with three layers and visualize it
inputs = keras.Input(shape=(784,))  # An input layer with 784 features (60,000 28X28 images from MNIST)
dense = layers.Dense(64, activation="relu")
x = dense(inputs) # This is like drawing an arrow from the input to the layer that we created
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(10)(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="mnist_model")  # Here we compile our lauers into a model
model.summary()

Model: "mnist_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
dense_7 (Dense)              (None, 64)                50240     
_________________________________________________________________
dense_8 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_9 (Dense)              (None, 10)                650       
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


In [25]:
# Can also plot the graph for the model
# keras.utils.plot_model(model, "my_model")

## Train the model

In [30]:
# Load the MNIST dataset for training
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype("float32") / 255
x_test = x_test.reshape(10000, 784).astype("float32") / 255

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),
    metrics=["accuracy"],
)

history = model.fit(x_train, y_train, batch_size=64, epochs=2, validation_split=0.2)

test_scores = model.evaluate(x_test, y_test, verbose=2)
print("Test loss:", test_scores[0])
print("Test accracy:", test_scores[1])

Train on 48000 samples, validate on 12000 samples
Epoch 1/2
48000/48000 [==============================] - 2s 50us/sample - loss: 0.3452 - accuracy: 0.9010 - val_loss: 0.1881 - val_accuracy: 0.9461
Epoch 2/2
48000/48000 [==============================] - 2s 39us/sample - loss: 0.1669 - accuracy: 0.9499 - val_loss: 0.1468 - val_accuracy: 0.9586
10000/10000 - 0s - loss: 0.1440 - accuracy: 0.9556
Test loss: 0.1439650827549398
Test accracy: 0.9556


## Save the model

In [33]:
# This saved file includes the:
# - model architecture
# - model weight values (that were learned during training)
# - model training config, if any (as passed to compile)
# - optimizer and its state, if any (to restart training where you left off)
model.save("local_model")

INFO:tensorflow:Assets written to: local_model\assets
